In [1]:
bs = 256
nw = 10
n_bars = 2
device_idx = 3
ds_dir = "data/prova/"

In [2]:
from torch.utils.data import Subset
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
from data import PolyphemusDataset
import torch


dataset = PolyphemusDataset(ds_dir, n_bars)
ds_len = len(dataset)

print('Dataset len:', len(dataset))

train_len = int(0.7 * len(dataset)) 
valid_len = int(0.1 * len(dataset))
test_len = len(dataset) - train_len - valid_len
tr_set, vl_set, ts_set = random_split(dataset, (train_len, valid_len, test_len))

trainloader = DataLoader(tr_set, batch_size=bs, shuffle=True, num_workers=nw)
validloader = DataLoader(vl_set, batch_size=bs, shuffle=False, num_workers=nw)

tr_len = len(tr_set)
vl_len = len(vl_set)
ts_len = len(ts_set)

print('TR set len:', len(tr_set))
print('VL set len:', len(vl_set))
print('TS set len:', len(ts_set))

Dataset len: 5149
TR set len: 3604
VL set len: 514
TS set len: 1031


In [3]:
torch.cuda.set_device(0)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#device = torch.device("cpu")
print("Device:", device)
print("Current device idx:", torch.cuda.current_device())

Device: cuda
Current device idx: 0


In [4]:
import math
import torch
from torch.optim import Optimizer


class ExpDecayLRScheduler():
    def __init__(
        self,
        optimizer: Optimizer,
        peak_lr: float,
        warmup_steps: int,
        final_lr_scale: float,
        decay_steps: int
    ):

        self.optimizer = optimizer
        self.peak_lr = peak_lr
        self.warmup_steps = warmup_steps
        self.decay_steps = decay_steps

        # Find the decay factor that is needed to reach the learning rate scale
        # after decay_steps steps
        self.decay_factor = -math.log(final_lr_scale) / self.decay_steps

        self.update_steps = 0

    def set_lr(self, optimizer, lr):
        for param_group in optimizer.param_groups:
                param_group['lr'] = lr
                
    
    def step(self):
        self.update_steps += 1
        
        if self.update_steps <= self.warmup_steps:
            self.lr = self.peak_lr
        else:
            steps_after_warmup = self.update_steps - self. warmup_steps
            self.lr = \
                self.peak_lr * math.exp(-self.decay_factor*steps_after_warmup)

        self.set_lr(self.optimizer, self.lr)

        return self.lr

In [5]:
parameters = {
    'training': {
        'batch_size': bs,
        'num_workers': nw,
        'ds_len': ds_len,
        'tr_len': tr_len,
        'vl_len': vl_len,
        'ts_len': ts_len,
    },
    'model': {
        'dropout': 0,
        'batch_norm': True,
        'gnn_n_layers': 8,
        'actsnn_n_layers': 2,
        'd': 512,
        'd_token': 230,
        'd_token_pitches': 131,
        'd_token_dur': 99,
        'n_bars': n_bars,
        'n_relations': 6,
        'n_tracks': 4,
        'resolution': 8,
        'max_simu_notes': 16
    },
    'scheduler': {
        'peak_lr': 1e-4,
        'final_lr_scale': 0.01,
        'warmup_steps': 8000,
        'decay_steps': 800000
    },
    'optimizer': {
        'betas': (0.9, 0.98),
        'eps': 1e-09,
        'lr': 5e-6
    },
    'beta_annealing': {
        'beta_update': True,
        'anneal_start': 40000,
        'beta_max': 0.01,
        'step_size': 0.001,
        'anneal_end': 500000
    }
}

In [6]:
from utils import print_params
import os
from model import VAE
import torch.optim as optim
from train import PolyphemusTrainer

print("Creating the model and moving it to the specified device...")

# Create model dir
models_dir = 'models_prova/'
model_name = 'prova'
model_dir = os.path.join(models_dir, model_name)
os.makedirs(models_dir, exist_ok=True) 
os.makedirs(model_dir, exist_ok=True)

# Creating the model
vae = VAE(**parameters['model'], device=device).to(device)

print_params(vae)
print()

# Creating optimizer and scheduler
optimizer = optim.Adam(vae.parameters(), **parameters['optimizer'])
scheduler = ExpDecayLRScheduler(
    optimizer=optimizer,
    **parameters['scheduler']
)

# Save parameters
params_path = os.path.join(model_dir, 'params')
torch.save(parameters, params_path)

print('--------------------------------------------------\n')

trainer = PolyphemusTrainer(
    model_dir,
    model=vae,
    optimizer=optimizer,
    lr_scheduler=scheduler,
    save_every=1,
    print_every=1,
    eval_every=18631,
    iters_to_accumulate=1,
    device=device,
    **parameters['beta_annealing']
)
trainer.train(trainloader, validloader=None, epochs=100)

Creating the model and moving it to the specified device...
+-------------------------------------------------------------+------------+
|                           Modules                           | Parameters |
+-------------------------------------------------------------+------------+
|                encoder.notes_pitch_emb.weight               |   33536    |
|                 encoder.notes_pitch_emb.bias                |    256     |
|                encoder.drums_pitch_emb.weight               |   33536    |
|                 encoder.drums_pitch_emb.bias                |    256     |
|                    encoder.dur_emb.weight                   |   25344    |
|                     encoder.dur_emb.bias                    |    256     |
|                    encoder.bn_npe.weight                    |    256     |
|                     encoder.bn_npe.bias                     |    256     |
|                    encoder.bn_dpe.weight                    |    256     |
|               

  0%|          | 0/15 [00:00<?, ?it/s]

Training on batch 1/15 of epoch 1/100 complete.
Elapsed time from start (h:m:s): 00:00:03.79
Losses:
{ 'acts': 0.64,
  'beta*kld': 0.0,
  'dur': 5.57,
  'kld': 66.0,
  'pitches': 5.56,
  'rec': 11.77,
  'tot': 11.77}
Accuracies:
{ 'dur': 0.0,
  'note': 0.0,
  'pitch': 0.0,
  'pitch_drums': 0.0,
  'pitch_non_drums': 0.01,
  's_acc': 0.14,
  's_f1': 0.25,
  's_precision': 0.14,
  's_recall': 1.0}

----------------------------------------

Saving model to disk...

The model has been successfully saved.
Training on batch 2/15 of epoch 1/100 complete.
Elapsed time from start (h:m:s): 00:00:06.78
Losses:
{ 'acts': 0.64,
  'beta*kld': 0.0,
  'dur': 5.51,
  'kld': 66.69,
  'pitches': 5.5,
  'rec': 11.65,
  'tot': 11.65}
Accuracies:
{ 'dur': 0.0,
  'note': 0.0,
  'pitch': 0.0,
  'pitch_drums': 0.0,
  'pitch_non_drums': 0.01,
  's_acc': 0.14,
  's_f1': 0.25,
  's_precision': 0.14,
  's_recall': 1.0}

----------------------------------------

Saving model to disk...

The model has been successful

KeyboardInterrupt: 